In [1]:
%matplotlib inline

import time
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from collections import defaultdict
from reachy_pyluos_hal.pycore import GateProtocol

data = defaultdict(lambda: defaultdict(list))

class GateHandler(GateProtocol):
    def handle_dxl_pub_data(self, register, ids, errors, values):
        for id, err, val in zip(ids, errors, values):
            data[id][register].append((err, val, time.time()))
    
    def handle_assert(self, msg):
        raise AssertionError(msg)

In [3]:
import sys

from serial import Serial
from serial.threaded import ReaderThread

if sys.platform == 'darwin':
    s = Serial('/dev/tty.usbserial-DN05NM0W', baudrate=1000000)
else:
    s = Serial('/dev/ttyUSB0', baudrate=1000000)
    s.set_low_latency_mode(True)

start = time.time()
with ReaderThread(s, GateHandler) as p:
    while time.time() - start < 60:
        p.send_keep_alive()
        time.sleep(1.0)

Connection made <ReaderThread(Thread-4, started daemon 123145633837056)>
Connection lost


In [4]:
ids = data.keys()
ids

dict_keys([10, 16, 11, 12, 13, 14, 15, 17])

In [5]:
for id in ids:
    print(data[id].keys())

dict_keys([36, 43])
dict_keys([43, 36])
dict_keys([36, 43])
dict_keys([36, 43])
dict_keys([36, 43])
dict_keys([36, 43])
dict_keys([36, 43])
dict_keys([36, 43])


In [6]:
for id in ids:
    err, pos, t = zip(*data[id][36])
    elapsed_time = t[-1] - t[0]
    print(f'Motor {id} received {len(pos)} pos update (in {elapsed_time:.1f}s)')
    dt = 1 / np.diff(t)
    print(f'\tAvg freq {np.mean(dt):.1f}Hz (Std={np.std(dt):.1f})')
    real_err = [e for e in err if e]
    print(f'\t Error: {real_err}')
    

Motor 10 received 13159 pos update (in 60.2s)
	Avg freq 246.1Hz (Std=79.5)
	 Error: []
Motor 16 received 13159 pos update (in 60.2s)
	Avg freq 246.1Hz (Std=79.7)
	 Error: []
Motor 11 received 13159 pos update (in 60.2s)
	Avg freq 245.9Hz (Std=79.2)
	 Error: []
Motor 12 received 13159 pos update (in 60.2s)
	Avg freq 246.0Hz (Std=79.4)
	 Error: []
Motor 13 received 13160 pos update (in 60.2s)
	Avg freq 245.9Hz (Std=79.3)
	 Error: []
Motor 14 received 13160 pos update (in 60.2s)
	Avg freq 246.1Hz (Std=79.6)
	 Error: []
Motor 15 received 13160 pos update (in 60.2s)
	Avg freq 246.0Hz (Std=79.6)
	 Error: []
Motor 17 received 13160 pos update (in 60.2s)
	Avg freq 245.9Hz (Std=79.3)
	 Error: []
